In [7]:
#%load_ext autoreload
#%autoreload 2

#import sys
#sys.path.insert(0,'..')

import os

import backtrader as bt
import backtrader.indicators as btind
import numpy as np
import scipy.signal as signal

from btgym import BTgymEnv, BTgymStrategy, BTgymDataset

from launcher import Launcher

In [ ]:
# GYM TEST ENV:
cluster_config = dict(
    host='127.0.0.1',
    port=12222,
    num_workers=6,
    num_ps=1,
    log_dir='./tmp/a3c_testing_gym',
)

env_config = dict(
    gym_id='Breakout-v0'
)


launcher = Launcher(
    cluster_config=cluster_config,
    env_config=env_config,
    train_steps=10000000,
    test_mode=True,
    model_summary_freq=50,
    episode_summary_freq=5,
    verbose=2
    
)

In [ ]:
launcher.run()


In [8]:
class MyStrategy(BTgymStrategy):
    """
    Example subclass of BT server inner computation startegy.
    """
    
    def set_datalines(self):
        self.state = np.zeros(self.p.state_shape)
        
        
    def get_state(self):
        """
        Computes featurized RL-ready environment observation state
        by applying continious wavelet transform to time-embedded vector
        of close-price gradients.
        """
        # Z-normalize raw inputs:
        mean = self.p.dataset_stat[1:3].values[0,:-1]
        std = self.p.dataset_stat[1:3].values[1,:-1]
        X = (self.raw_state - mean) / std
        #X = self.raw_state
        
        # Prepare parameters:
        Tau = 5
        max_cwt_scale = self.p.state_shape[1]
        cwt_width = np.linspace(Tau, max_cwt_scale + Tau - 1, max_cwt_scale) # scale of wavelet transdorm [n]
        
        sigmoid = lambda x: 1/(1 + np.exp(-x))
        
        # 'gamma'-like signal hyperparameter
        # for our signal to be in about [-5,+5] range before passing it to sigmoid;
        # tweak it by hand to add/remove "contrast":
        T = 2.5e+4
        
        # Get vector of gradients of last [n] prices:
        X = np.gradient(X, axis=0) * T
        
        # Use close price:
        channel = 3
        # Compute continious wavelet transform using Ricker wavelet, get [n,m,1]-dim. matrix:
        self.state = signal.cwt(X[:, channel], signal.ricker, cwt_width).T[:, :, None]
        
        # Squash values in [0,1]:
        return sigmoid(self.state)
    

    def R(self, val, val_start, R_max, DD_max):
        """
        Piecewise-linear normalized reward:
        val > 0         - values scalar or vector;
        val_start > 1   - start value, scalar;
        0 < R_max < 1   - maximum gain, percent/100,
        0 < DD_max < 1  - maximum draw-down, percent/100,
                          scalar (minimal acceptable value: = val_start*(1-DD_max) );
        returns vector in [-1,+1] range :
        """

        f_neg = np.asarray((val/val_start + DD_max - 1) / DD_max - 1)
        f_neg[f_neg > 0] = 0 
        f_neg[f_neg < -1] = -0.999
        f_pos = np.asarray((val/val_start - 1) / R_max)
        f_pos[f_pos < 0] = 0
        f_pos[f_pos > 1] = 1 

        return (f_neg + f_pos) 
    
    def get_reward(self):
        """
        Defines reward as [0,1]-bounded linear function of current to initial portfolio value ratio.
        """
        return float(self.R(val=self.env.broker.get_value(),
                            val_start=self.env.broker.startingcash,
                            R_max=self.p.target_call/100,
                            DD_max=self.p.drawdown_call/100,
                           )
                    )/2 +.5  # shift reward to [0,1] range
            
# Set backtesting engine parameters:

state_shape = (40, 40, 1)

MyCerebro = bt.Cerebro()
MyCerebro.addstrategy(MyStrategy,
                      state_shape=state_shape,
                      drawdown_call=10, # in percent
                      target_call=5,
                      state_low=0,
                      state_high=1,
                      skip_frame=6,
                     )

# Set leveraged account:
MyCerebro.broker.setcash(2000)
MyCerebro.broker.setcommission(commission=0.0001, leverage=10.0)
MyCerebro.broker.set_shortcash(False)
MyCerebro.addsizer(bt.sizers.SizerFix, stake=10000,)


MyCerebro.addanalyzer(bt.analyzers.DrawDown)

# Provide data:
MyDataset = BTgymDataset(
    filename='../data/DAT_ASCII_EURUSD_M1_2016.csv',
    start_weekdays=[0, 1, 2,],
    episode_len_days=1,
    episode_len_hours=23,
    episode_len_minutes=0,
    start_00=False,
    time_gap_hours=2,
)
env_config = dict(
    dataset=MyDataset,
    engine=MyCerebro,
    render_agent_as_image=True,
    render_ylabel='Z-norm/CWT',
    render_size_episode=(12,8),
    render_size_human=(8, 3.5),
    render_size_agent=(10, 5),
    render_dpi=75,
    port=5000,
    data_port=4999,
    connect_timeout=60,
    verbose=0,
)
cluster_config = dict(
    host='127.0.0.1',
    port=12222,
    num_workers=2,
    num_ps=1,
    log_dir='./tmp/a3c_testing',
)
launcher = Launcher(
    cluster_config=cluster_config,
    env_class=BTgymEnv,
    env_config=env_config,
    test_mode=False,
    train_steps=5000,
    model_summary_freq=10,
    episode_summary_freq=2,
    env_render_freq=10,
    verbose=1
    
)

[2017-08-14 19:58:34,408] port 12222 cleared


In [ ]:
print(launcher.kwargs, '\n\n')
print(launcher.env_config)
print(launcher.cluster_config)
print(launcher.cluster_spec)
for config in launcher.workers_config_list:
    print('============')
    for k, v in config.items():
        print('{}:\n{}\n'.format(k, v))
    

In [9]:
launcher.run()

[2017-08-14 19:58:36,348] parameters_server started.
[2017-08-14 19:58:36,353] tf.server started.
[2017-08-14 19:58:36,356] making environment.
[2017-08-14 19:58:36,357] worker_0 is data_master: True
[2017-08-14 19:58:41,179] connecting to the parameter server... 


INFO:tensorflow:Restoring parameters from ./tmp/a3c_testing/train/model.ckpt-0


[2017-08-14 19:58:41,182] Restoring parameters from ./tmp/a3c_testing/train/model.ckpt-0
[2017-08-14 19:58:41,355] tf.server started.
[2017-08-14 19:58:41,359] making environment.
[2017-08-14 19:58:41,360] worker_1 is data_master: False


INFO:tensorflow:Starting standard services.


[2017-08-14 19:58:44,642] Starting standard services.


...

[2017-08-14 19:58:44,653] Starting queue runners.


INFO:tensorflow:global/global_step/sec: 0


[2017-08-14 19:58:44,654] global/global_step/sec: 0
[2017-08-14 19:58:44,685] worker_0: trainer synch`ed
[2017-08-14 19:58:44,687] worker_0: trainer started
[2017-08-14 19:58:44,689] worker_0: starting training at step: 0
[2017-08-14 19:58:45,223] connecting to the parameter server... 


INFO:tensorflow:Starting queue runners.


[2017-08-14 19:58:45,895] Starting queue runners.
[2017-08-14 19:58:45,916] worker_1: trainer synch`ed
[2017-08-14 19:58:45,918] worker_1: trainer started
[2017-08-14 19:58:45,926] worker_1: starting training at step: 80
/Users/muzikin/anaconda/envs/tensorforce/lib/python3.6/site-packages/ipykernel_launcher.py:27: RuntimeWarning: overflow encountered in exp
/Users/muzikin/anaconda/envs/tensorforce/lib/python3.6/site-packages/ipykernel_launcher.py:27: RuntimeWarning: overflow encountered in exp
Exception in thread Thread-4:
Traceback (most recent call last):
  File "/Users/muzikin/anaconda/envs/tensorforce/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/Users/muzikin/Yandex.Disk.localized/work/btgym/examples/a3c/a3c.py", line 100, in run
    self._run()
  File "/Users/muzikin/Yandex.Disk.localized/work/btgym/examples/a3c/a3c.py", line 120, in _run
    self.queue.put(next(rollout_provider), timeout=600.0)
  File "/Users/muzikin/Yandex.Disk.localized/wor

In [ ]:
def func1(max_step):
    step = 0
    done = False
    
    def func2(max_step):
        nonlocal step
        nonlocal done
        step +=1
        if step == max_step:
            step = 0
            done = True
        return step
    
    for i in range(20):
        done = False
        print(func2(max_step), step, done)
        


func1(7)
            

In [ ]:
a = dict()
a.update({'b': 2, 'c':4})
a